# data clean

In [1]:
import numpy as np
import pandas as pd

### 识别税收不遵从

In [7]:
deregulation = pd.concat([
    pd.read_excel('RESSET_DEREGULATION_EXT_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_DEREGULATION_EXT_2.xls', engine='xlrd')
], ignore_index=True)
deregulation['year'] = deregulation['信息发布日期_InfoPubDt'].dt.year - 1


In [8]:
# 检查处理原因和处分措施中是否包含关键词
pattern = '|'.join(['税'])
tax_noncomp = deregulation[
    deregulation['涉及法规_InvolStatute'].astype(str).str.contains(pattern, regex=True) # |
    # deregulation['处分措施_Pena'].astype(str).str.contains(pattern, regex=True)
]
tax_noncomp = tax_noncomp.drop_duplicates(subset=['year', '上市公司代码_ComCd'])[['上市公司代码_ComCd','最新公司全称_LComNm', 'year']]
tax_noncomp['noncompliance'] = 1

### 财报指标

In [4]:
finratio = pd.concat([
    pd.read_excel('RESSET_FINRATIO_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_2.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_3.xls', engine='xlrd')
], ignore_index=True)
finratio['year'] = finratio['截止日期_Enddt'].dt.year

In [5]:
finratio['截止日期_Enddt']

0        2015-12-31
1        2016-12-31
2        2017-12-31
3        2018-12-31
4        2019-12-31
            ...    
119204   2024-12-31
119205   2022-12-31
119206   2023-12-31
119207   2022-12-31
119208   2023-12-31
Name: 截止日期_Enddt, Length: 119209, dtype: datetime64[ns]

In [6]:
finratio['年份'] = finratio['截止日期_Enddt'].dt.year